In [ ]:
'''
Author: Sriram Yarlagadda + George Nakhleh
Date Created: 9/12/2016
Note: Please use Python 2.7
'''
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats import proportion
from scipy import stats
%matplotlib inline
os.chdir('/Users/gangliu/Desktop/project wyc')#location in Mac

varLst = ['CLAM', 'ACDF', 'LOS', 'SHORT', 'LONG', 'ALIF', 'PLIF', 'COMBINED','ileus_NPOA','SHORT_ILEUS', \
          'LONG_ILEUS', 'ALIF_ILEUS', 'PLIF_ILEUS', 'DIED','MI_NPOA','Sepsis_NPOA','PE_NPOA',\
          'COMBINED_SHORT', 'COMBINED_LONG', 'COMBINED_ILEUS', 'COMBINED_SHORT_ILEUS','COMBINED_LONG_ILEUS',\
          'ACDF_ILEUS','CLAM_ILEUS']
CAData = pd.read_stata('SPINE_ILEUS_CA_REVISED.dta', columns=varLst)

In [33]:
CAData.describe()

,CLAM,ACDF,LOS,SHORT,LONG,ALIF,PLIF,COMBINED,ileus_NPOA,SHORT_ILEUS,...,MI_NPOA,Sepsis_NPOA,PE_NPOA,COMBINED_SHORT,COMBINED_LONG,COMBINED_ILEUS,COMBINED_SHORT_ILEUS,COMBINED_LONG_ILEUS,ACDF_ILEUS,CLAM_ILEUS
count,345067.000000,345067.000000,345010.000000,345067.000000,345067.000000,345067.000000,345067.000000,345067.000000,345067.000000,345067.000000,...,345067.000000,345067.000000,345067.0,345067.000000,345067.000000,345067.000000,345067.000000,345067.000000,345067.000000,345067.000000
mean,0.043823,0.195733,7.709695,0.459282,0.121730,0.084940,0.061374,0.010923,0.443998,0.010975,...,0.008416,0.044522,0.0,0.074049,0.014551,0.001078,0.005373,0.001837,0.000606,0.000484
std,0.204741,0.397012,79.797306,0.498372,0.327466,0.279424,0.240172,0.103546,0.496490,0.103827,...,0.091449,0.206195,0.0,0.262318,0.119164,0.032803,0.072964,0.042796,0.024598,0.021989
min,0.000000,0.000000,-6666.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,358.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
len(CAData.ACDF)
    

308119

##### for i in range len(CAData):
#   if CAData.CLAM

In [35]:
np.sum(np.array(CAData < 0), 0)

array([ 0,  0, 48,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 19,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0])

In [36]:
np.sum(np.array(pd.isnull(CAData)),0)

array([ 0,  0, 57,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 22,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0])

In [37]:
np.sum(np.array(CAData.LOS <= 1), 0)

36850

In [38]:
for feature in CAData.columns: 
    CAData = CAData[CAData[feature] >= 0]
    if feature == 'LOS':
        CAData = CAData[CAData.LOS >= 2]#去掉 LOS 小于1的

In [39]:
CAData.shape

(308119, 24)

In [40]:
ct_PvA = pd.crosstab(CAData.ACDF_ILEUS, CAData.MI_NPOA); ct_PvA

MI_NPOA,0.0,1.0
ACDF_ILEUS,,
0.0,305052,2862
1.0,203,2


In [41]:
pd.crosstab(CAData.ACDF_ILEUS, CAData.MI_NPOA).apply(lambda x: x/x.sum(), 0)

MI_NPOA,0.0,1.0
ACDF_ILEUS,,
0.0,0.999335,0.999302
1.0,0.000665,0.000698


In [42]:
print ('Odds ratio:', stats.fisher_exact(ct_PvA)[0])

Odds ratio: 1.05011824726


In [44]:
ct_Pvb = pd.crosstab(CAData.CLAM_ILEUS, CAData.MI_NPOA); ct_Pvb

MI_NPOA,0.0,1.0
CLAM_ILEUS,,
0.0,305091,2861
1.0,164,3


In [45]:
print ('Odds ratio:', stats.fisher_exact(ct_Pvb)[0])

Odds ratio: 1.95069308872


In [122]:
CADataSub = CAData[(CAData.ACDF+ CAData.CLAM) >=1]; CADataSub.shape

(48605, 24)

In [171]:
def g(x, y):
    return (x + y)//2

In [172]:
CAData.Combin = g(CAData.ACDF, CAData.CLAM)

In [173]:
ct_PvA = pd.crosstab(CAData.MI_NPOA, CAData.Combin); ct_PvA

col_0,0.0,1.0
MI_NPOA,,
0.0,301647,3608
1.0,2840,24


In [174]:
print ('Odds ratio:', stats.fisher_exact(ct_PvA)[0])

Odds ratio: 0.706521501515
